In [ ]:
import os
import tensorflow
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import locale
import sys
!pip install hdf5storage
import hdf5storage as hdf5

sys.path.insert(0,"/content/drive/My Drive/Hyperspectral")

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
os.chdir("/content/gdrive/MyDrive/Hyperspectral/")
print(os.getcwd())
!pip install hdf5storage
import datasets
import models
import cv2

     |████████████████████████████████| 53 kB 1.4 MB/s 
Mounted at /content/gdrive
/content/gdrive/MyDrive/Hyperspectral


In [ ]:
# load firmness
#inputPath = "c:\\users\\nalina\\Documents\\tomato_firmness\\hsdata\\firmness.csv"
inputPath = "/content/gdrive/MyDrive/Hyperspectral/hsdata/firmness.csv"
df = datasets.load_firmness_attributes(inputPath)


In [ ]:
#load the hyperspectral images of tomatoes (31 channels) and scale the pixel 
inputPath = "/content/gdrive/MyDrive/Hyperspectral/hsdata/"
#images = datasets.load_produce_images(df_sub, inputPath)
inputImages = []

for i in df.index.values:
  spectral = hdf5.loadmat(inputPath + i + ".jpeg.mat")
  #img = spectral["cube"][226:482,226:482]
  img = spectral["cube"]
  img = cv2.resize(img, (256, 256))
  #img_reshaped = img.reshape(img.shape[0], -1)
  #np.savetxt((img_path + i + ".txt"), img_reshaped, fmt='%d')
  inputImages.append(img)

In [ ]:
inputImages = np.array(inputImages)
inputImages.shape

(494, 256, 256, 31)

In [ ]:
#img_path = "/content/gdrive/MyDrive/Hyperspectral/hsdata/images/" 
#cnnImages = []
#for i in df.index.values:
  # retrieving data from file.
  #arr = np.loadtxt(img_path + i + ".txt")
  #This loadedArr is a 2D array, so convert it to the original array shape.reshaping to get original matrice with original shape.
  #original_arr = arr.reshape(arr.shape[0], arr.shape[1] // 31, 31)
  #original_arr = original_arr / 255.0
  #cnnImages.append(original_arr) 

In [ ]:
split = train_test_split(df, inputImages, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

trainY = np.log(trainAttrX["firmness"])
testY = np.log(testAttrX["firmness"])

In [ ]:
# create our Convolutional Neural Network and then compile the model
# using mean absolute percentage error as our loss, implying that we
# seek to minimize the absolute percentage difference between our
# predicted firmness and the actual firmness
model = models.create_cnn(256, 256, 31, regress=True)
opt = Adam(lr=1e-4, decay=1e-3 / 200)
model.compile(loss="mean_squared_error", optimizer=opt)

history = model.fit(x=trainImagesX, y=trainY, validation_data=(testImagesX, testY), epochs=200, batch_size=32) #try epochs=200 if this works, error goes down until 125

#firmness predictions for test data
#firmness_preds = model.predict(testImagesX)

#compute the difference between the firmness predictions and the actual firmness values, then compute the percentage 
#difference and the absolute percentage difference
#diff = firmness_preds.flatten() - testY
#percent_diff = (diff / testY) * 100
#absPercentDiff = np.abs(percent_diff)

#mean and std deviation of absolute percentage difference
#mean = np.mean(absPercentDiff)
#std = np.abs(absPercentDiff)
#######################################

#print("Avg. firmness " + str(df["firmness"].mean()))
#print("Std. Dev. firmness " + str(df["firmness"].std()))

#print("Mean and std. dev. of predictions " + str(mean) + " " + str(std))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/125
12/12 [==============================] - 21s 2s/step - loss: 5.4359 - val_loss: 2.1788
Epoch 2/125
12/12 [==============================] - 19s 2s/step - loss: 4.8330 - val_loss: 3.7326
Epoch 3/125
12/12 [==============================] - 19s 2s/step - loss: 4.6119 - val_loss: 4.3724
Epoch 4/125
12/12 [==============================] - 19s 2s/step - loss: 3.8719 - val_loss: 3.8558
Epoch 5/125
12/12 [==============================] - 18s 2s/step - loss: 3.4610 - val_loss: 3.2037
Epoch 6/125
12/12 [==============================] - 19s 2s/step - loss: 3.6857 - val_loss: 2.7767
Epoch 7/125
12/12 [==============================] - 19s 2s/step - loss: 3.5784 - val_loss: 2.4874
Epoch 8/125
12/12 [==============================] - 19s 2s/step - loss: 3.0913 - val_loss: 2.1206
Epoch 9/125
12/12 [==============================] - 19s 2s/step - loss: 2.8326 - val_loss: 1.9382
Epoch 10/125
12/12 [==============================] - 19s 2s/step - loss: 3.0589 - val_loss: 1.8398
Epoch 11/

In [ ]:
import matplotlib.pyplot as plt

print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: ignored

In [ ]:
validPath = "/content/gdrive/MyDrive/Hyperspectral/hsdata/valid_firmness.csv"
vdf = datasets.load_firmness_attributes(validPath)

In [ ]:
#load the hyperspectral images of tomatoes (31 channels) and scale the pixel 
inputPath = "/content/gdrive/MyDrive/Hyperspectral/hsdata/"
#images = datasets.load_produce_images(df_sub, inputPath)
validImages = []

for i in df.index.values:
  spectral = hdf5.loadmat(inputPath + i + ".jpeg.mat")
  #img = spectral["cube"][226:482,226:482]
  img = spectral["cube"]
  img = cv2.resize(img, (256, 256))
  #img_reshaped = img.reshape(img.shape[0], -1)
  #np.savetxt((img_path + i + ".txt"), img_reshaped, fmt='%d')
  validImages.append(img)

In [ ]:
import pandas as pd
#testY = testAttrX["firmness"] / maxFirmness
#firmness predictions for test data
valid_firmness_preds = model.predict(validImages)
vdf['preds'] = valid_firmness_preds.flatten()
testY.to_csv("/content/gdrive/MyDrive/Hyperspectral/hsdata/valid_preds.csv", index=False)

In [ ]:
d = {'train_loss': history.history['loss'], 'test_loss': history.history['val_loss']}
model_acc = pd.DataFrame(d)
model_acc.to_csv("/content/gdrive/MyDrive/Hyperspectral/hsdata/model_acc_256rs_125epochs_v3.csv", index=False)

In [ ]:
#load the hyperspectral images of tomatoes (31 channels) and scale the pixel to 128 or 64 for NN since only a single dimension can be input 128*128*31 or 64*64*31
inputPath = "/content/gdrive/MyDrive/Hyperspectral/hsdata/"
#images = datasets.load_produce_images(df_sub, inputPath)
inputNNImages = []
#img_path = "/content/gdrive/MyDrive/Hyperspectral/hsdata/images/" 
for i in df.index.values:
  spectral = hdf5.loadmat(inputPath + i + ".jpeg")
  #img = spectral["cube"][226:482,226:482]
  img = spectral["cube"]
  img = cv2.resize(img, (128, 128))
  #img_reshaped = img.reshape(img.shape[0], -1)
  #np.savetxt((img_path + i + ".txt"), img_reshaped, fmt='%d')
  inputNNImages.append(img)

In [ ]:
inputNNImages = np.array(inputNNImages)

In [ ]:
inputNNImages.shape

In [ ]:
split = train_test_split(df, inputNNImages, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

trainY = np.log(trainAttrX["firmness"])
testY = np.log(testAttrX["firmness"])

In [ ]:
#flatten the trainImagesX and testImagesX to convert from 3 dimensions to 1 dimension
nn_trainx = trainImagesX.reshape(trainImagesX.shape[0],-1)
nn_testx = testImagesX.reshape(testImagesX.shape[0],-1)
print(nn_trainx.shape)


In [ ]:
# create our Neural Network and then compile the model
# using mean absolute percentage error as our loss, implying that we
# seek to minimize the absolute percentage difference between our
# predicted firmness and the actual firmness
model = models.create_mlp(507904, regress=True)
opt = Adam(lr=1e-4, decay=1e-3 / 200)
model.compile(loss="mean_squared_error", optimizer=opt)

nn_history = model.fit(x=nn_trainx, y=trainY, validation_data=(nn_testx, testY), epochs=125, batch_size=32) #try epochs=200 if this works

#firmness predictions for test data
nn_firmness_preds = model.predict(nn_testx)

#compute the difference between the firmness predictions and the actual firmness values, then compute the percentage 
#difference and the absolute percentage difference
#nn_diff = firmness_preds.flatten() - testY
#nn_percent_diff = (nn_diff / testY) * 100
#nn_absPercentDiff = np.abs(nn_percent_diff)

#mean and std deviation of absolute percentage difference
#nn_mean = np.mean(nn_absPercentDiff)
#nn_std = np.abs(nn_absPercentDiff)

In [ ]:
import matplotlib.pyplot as plt

print(nn_history.history.keys())
# summarize history for accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import pandas as pd
#testY = testAttrX["firmness"] / maxFirmness
#firmness predictions for test data
nn_firmness_preds = model.predict(nn_testx)
testY = pd.DataFrame(testY)
testY['preds'] = nn_firmness_preds.flatten()
testY.to_csv("/content/gdrive/MyDrive/Hyperspectral/hsdata/nn_test_preds_256rs_125epochs_v2.csv", index=False)

In [ ]:
d = {'train_loss': nn_history.history['loss'], 'test_loss': nn_history.history['val_loss']}
model_acc = pd.DataFrame(d)
model_acc.to_csv("/content/gdrive/MyDrive/Hyperspectral/hsdata/nn_model_acc_256rs_125epochs.csv", index=False)